In [1]:

import nltk
import numpy as np
from textblob import TextBlob
import pandas as pd 
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import KFold, StratifiedKFold
#Load libraries needed for classification 
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB,GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import ComplementNB
#from bayes.classifiers import ComplementNB
#from naiveBayesClassifier.classifier import Classifier
from sklearn import svm, datasets
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.util import ngrams
from nltk.corpus import wordnet as omw # open multilingual wordnet
from nltk.stem.isri import ISRIStemmer



In [2]:
bArabic=True

In [3]:
import warnings;
warnings.filterwarnings('ignore');

In [4]:
# A.read the course names training data & apply preprocessing into a data frame
# A.1 upload  Train data file names
path =r'D:\MahmoudOut\Final Project\CoursesAnalysis-master'
ds1_train=pd.read_excel(path+"\CoursesTrainData.xlsx")

In [5]:
#get Arabic sentences only
import re

def ArabicOnly(text):
    #print (text)
    result = re.sub(r'[^ء-ي]',' ',str(text)) #arabic
    if len(result.strip()) >0:
        return 'T'
    else:
        return 'F'
#ds1_train['ArabicOnly'] =ds1_train['TrCourse'].apply(lambda x : ArabicOnly(x))


In [6]:
#ds1_train = ds1_train[ds1_train.ArabicOnly =='T']
#len(ds1_train)

In [7]:
from string import punctuation

# Arabic diacritics
arabic_punct = ''' ` ÷ × ؛ < > _ ( ) * & ^ % ] [ ـ ، / : " ؟ . , ' { } ~ ¦ + | !  ”  …  “ –   ـ  -'''
arabic_diacritics = '''  َ     ُ       ِ      ّ       ً      ٌ       ٍ      ْ     '''

arabic_punct = arabic_punct.split()

arabic_diacritics = arabic_diacritics.split()

# Arabic , English punctuations and dicritis 
punctuations = set( arabic_punct  + arabic_diacritics )|set(punctuation)

# Arabic stopwords. This list are obtained from http://www.ranks.nl/stopwords/arabic 
asw = open(path+'/ArabicStopwords.txt', encoding="utf8").read()

arabicStopWords =  asw.split() #+ aswUinicode.split()

asw2 = ''

#english stopwords
ext_st_word_list=[]
ext_st_word=pd.read_csv(path+'\stopwords.csv')
ext_st_word_list=ext_st_word['Stop words']
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(ext_st_word_list)

# Arabic  stopwords
all_stopwords = set(arabicStopWords+ stopwords)


In [8]:
# remove punctcutions
def remove_punct(word):
    for c in word: return ''.join(ch for ch in word if not ch in punctuations) # remove punctuation
###################################################################################

In [9]:
# remove arabic diacritics
def remove_diacritics(text):
    arstemmer = ISRIStemmer()
    result = arstemmer.norm(text, num=1) #  remove diacritics which representing Arabic short vowels
    return result
###################################################################################

In [10]:
# takes a word list and returns the root for each Arabic words
def getRootAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list: result.append(arstemmer.stem(word))
    return result#' '.join(result)

In [11]:
from nltk.stem import WordNetLemmatizer
# takes a word list and returns the root for each english words
def getRootEn(word_list):
    result = []
    lemmatizer = WordNetLemmatizer()
    for word in word_list: result.append(lemmatizer.lemmatize(word, pos='v'))
    return result#' '.join(result)

In [12]:
arstemmer = ISRIStemmer()
print(arstemmer.pre32('الوقت'))
print(arstemmer.suf32('بنائون'))
print(arstemmer.waw('وورد'))
print(arstemmer.norm('أحمد', num=2))


وقت
بنائ
ورد
احمد


In [13]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [14]:
print(normalize_arabic('ؤ'))

ء


In [15]:
# Arabic light stemming for Arabic text
# takes a word list and perform light stemming for each Arabic words
def lightStemAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list:
        word = arstemmer.norm(word, num=1)  #  remove diacritics which representing Arabic short vowels  
        if not word in arstemmer.stop_words:   # exclude stop words from being processed
            word = arstemmer.pre32(word)        # remove length three and length two prefixes in this order
            word = arstemmer.suf32(word)        # remove length three and length two suffixes in this order
            word = arstemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            word = arstemmer.norm(word, num=2)       # normalize initial hamza to bare alif
            word = normalize_arabic(word)
        result.append(word)
    return result#' '.join(result)

In [16]:
from nltk.stem.porter import PorterStemmer
# English light stemming for English text
def lightStemEn(word_list):
    result = []
    enStemmer = PorterStemmer()
    for word in word_list:       
        word = enStemmer.stem(word)
        result.append(word)
    return result#' '.join(result)

In [17]:
#remove longation
def removeLongation(word_list):
    result = []
    for word in word_list:
        #remove longation
        p_longation = re.compile(r'(.)\1+')
        subst = r"\1"
        word = re.sub(p_longation, subst, word) 
        result.append(word)
    return result

In [18]:
# takes a string of text and returns the word list (tonkized words)
# processing includes: removing diacritics and punctcutions, removing stopwords, and tokenizing
# Stemmning
# Get Root
def process_text(text, removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=True, IsArabic=False):
    if IsArabic :
        text = remove_diacritics(text)# remove arabic diacritics
        
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    if IsArabic :
        word_list = removeLongation(word_list) #remove longation
        
    if removePunct:
        word_list = [ w for w in word_list if not w in punctuations ]
        word_list = [ remove_punct(w) for w in word_list ]
        
    if removeSW: word_list = [ w for w in word_list if not w in all_stopwords ]
    if removeNum: word_list = [ w for w in word_list if not w.isdigit() ]
        
    if Stem: #stemming
        if IsArabic :
            word_list = lightStemAr(word_list)  #arabic stemmer
        else :
            word_list = lightStemEn(word_list) #english stemmer
            
    if Root: #get Root
        if IsArabic :
            word_list = getRootAr(word_list) 
        else:
            word_list = getRootEn(word_list) 
       
    return ' '.join(word_list)
###################################################################################

In [19]:
#ds1_train['preprocessingWithStem'] =ds1_train['TrCourse'].apply(lambda x : process_text(x,True,True,False,True,False))

In [20]:
#ds1_train['preprocessingWithRoot'] =ds1_train['TrCourse'].apply(lambda x : process_text(x,True,True,False,True,True))

In [21]:
def PreProcessing(data= pd.DataFrame, removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=True, IsArabic=False):
    data['ArabicOnly'] =data['TrCourse'].apply(lambda x : ArabicOnly(x))
    if IsArabic : #get arabic records 
        data = data[data.ArabicOnly =='T']
    else : #get english records
        data = data[data.ArabicOnly =='F']
    
    result= pd.DataFrame()
    result['CourseName']  = data['TrCourse'].apply(lambda x : process_text(x,removePunct,removeSW,removeNum,Stem,Root,IsArabic))
    result['Category'] = data['Category']
    return result
    


In [22]:
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=False, IsArabic=bArabic)
preProcessed

,CourseName,Category
27,توعيه عامل مخاطر كهربيه,Awareness (AW)
31,اجراء سلامه صحه مهنيه لعامل مرافق اثناء تعامل ...,Awareness (AW)
33,توعيه عامل باعداد درفله تعليم سلامه صحه مهنيه ...,Awareness (AW)
34,توعيه عامل بخط انتاج اطوال تعليم سلامه صحه مهن...,Awareness (AW)
35,توعيه عامل تسليم اتباع تعليم سلامه صحه مهنيه,Awareness (AW)
38,تعليم مراقبه اداء بيءي بمعامل جوده inwi,Awareness (AW)
39,توعيه عامل تعليم سلامه صحه مهنيه لانشطه مراقبه...,Awareness (AW)
67,احكام قان تامين اجتماعيه جديد,Functional Programs (FN)
89,تعليم مراقبه اداء بيءي لفرن اعاده تسخ enwi,Awareness (AW)
91,استخدام موقع الكتروني خاص باداره توكيد جوده بي...,Awareness (AW)


In [23]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.pipeline import FeatureUnion
from scipy.sparse  import coo_matrix
from sklearn.metrics import classification_report
import csv
from io import StringIO
import re

In [24]:
'''
pip install fim
'''
import pandas as pd
import fim
from mlxtend.frequent_patterns import association_rules
import math
from time import time

def generate_association_rules(dataset, algo, min_support, metric, min_threshold):
    '''
    dataset: list of lists for transactions
    alg: apriori or fpgrowth
    min_support: min support for generating the frequent itemsets. its range from 0.0 to 1.0
    metric: criteria for generating association rules (e.g. "confidence")
    min_threshold: metric min needed value
    '''
    #step1: convert min_support to integer value
    min_support = int(math.ceil(min_support*len(dataset)))
    #step2: generate frequent itemsets with apriori algorithm
    t1 = time()
    if algo == 'apriori':
        frequent_itemsets = fim.apriori(dataset, supp=-1*min_support)
    else:
        frequent_itemsets = fim.fpgrowth(dataset, supp=-1*min_support)
    t2 = time()
    time_elapsed = (t2-t1)/60.0 #in minutes
    #step3: transform frequent itemsets from a dictionary to list of dictionaries
    frequent_itemsets = [{'support':v*1.0/len(dataset), 'itemsets': list(k)} for k,v in frequent_itemsets]
    #step3: transform frequent itemsets from list of dictionaries to dataframe
    frequent_itemsets = pd.DataFrame(frequent_itemsets)
    
    #step4: generate association rules from the frequent itemsets
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)
    return (frequent_itemsets, rules, time_elapsed)

In [25]:
preProcessed["CourseName"]

27                               توعيه عامل مخاطر كهربيه
31     اجراء سلامه صحه مهنيه لعامل مرافق اثناء تعامل ...
33     توعيه عامل باعداد درفله تعليم سلامه صحه مهنيه ...
34     توعيه عامل بخط انتاج اطوال تعليم سلامه صحه مهن...
35          توعيه عامل تسليم اتباع تعليم سلامه صحه مهنيه
38               تعليم مراقبه اداء بيءي بمعامل جوده inwi
39     توعيه عامل تعليم سلامه صحه مهنيه لانشطه مراقبه...
67                         احكام قان تامين اجتماعيه جديد
89            تعليم مراقبه اداء بيءي لفرن اعاده تسخ enwi
91     استخدام موقع الكتروني خاص باداره توكيد جوده بي...
100    توعيه بيان تحليل اصاب سابقه لعامل بقسم مراقبه ...
102                                   تعامل قان عمل جديد
154    تعليم سلامه صحه مهنيه لعامل انتاج صيانه وكيفيه...
156               تعليم سلامه صحه مهنيه لعامل صيانه hswi
157     تعليم سلامه صحه مهنيه واجب اتباع مطعم بوفيه hswi
160    تعليم سلامه صحه مهنيه خاصه بمهم وقايه شخصيه عم...
161    مكافحه حراءق داخل مباني استخدام اجهزه اطفاء تم...
162       تعليم طوارء لمعامل مر

In [26]:
def ConvertToRules(dataset , labels, IsArabic=False):
    
    preProcessed_ItemSet = dataset.apply(lambda x : x.split(' ')) + labels.apply(lambda x : ["CAT:"+x])
    if IsArabic :
        min_support = 0.03
    else :
        min_support = 0
    (frequent_itemsets, rules, time_elapsed) = generate_association_rules(preProcessed_ItemSet, algo = 'apriori', min_support = min_support, metric="confidence", min_threshold=0.5)
    
    rules['IsCategory'] = rules['consequents'].apply(lambda x : len(x)==1 and str(x).find('CAT:')>=0)
    rules = rules[rules.IsCategory]
    rules = rules.sort_values("support", ascending=False).groupby(['antecedents'], as_index=False).first()
    #convert from tubels (item1,item2) to text item1 item2
    return rules["antecedents"].apply(lambda x :' '.join(word for word in x ).strip()),rules["consequents"].apply(lambda x : re.sub(r'CAT:','',str(''.join(word for word in x ))).strip())
    
    
    

In [27]:
#ConvertToRules(preProcessed["CourseName"] , preProcessed["Category"])

In [28]:
#train Data    
def Train(data , vectorizer, clfr  ,n_splits, fileName, IsApriori, IsArabic=False ):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    allLabels = []#pd.DataFrame()
    allPredicted = []#pd.DataFrame()
    nIndex = 0
    for train_idx, val_idx in kf.split(data["CourseName"], data["Category"]):
        nIndex = nIndex+1 
        train_data = data["CourseName"].iloc[train_idx]
        train_label = data["Category"].iloc[train_idx]
        val_data = data["CourseName"].iloc[val_idx]
        val_label = data["Category"].iloc[val_idx]
        #val_data.fillna(0)
        #train_data.fillna(0)
        if IsApriori == 'T': #Apriori algorithm
            train_data,train_label = ConvertToRules(train_data,train_label,IsArabic) 
            
        
        xtrain = vectorizer.fit_transform(train_data)
        xval = vectorizer.transform(val_data)
        #xtrain.fillna(0)
        #xval.fillna(0)
        clfr.fit(xtrain,train_label)
        predicted = clfr.predict(xval)
        allLabels.extend(val_label)
        allPredicted.extend(predicted)
        #print('---------------------Iteration '+str(nIndex)+'----------------------')
        #print(classification_report(val_label,predicted))
    print('------------'+fileName+'---------Average '+str(n_splits)+' Folds----------------------')
    print(classification_report(allLabels,allPredicted))


In [29]:
import os
import nltk
import gensim
from gensim.models import KeyedVectors

class EmbeddingVectorizer:
    def __init__(self, isArabic=False):
        # load the model
        if isArabic:
            self.embeddings = gensim.models.Word2Vec.load(path+'\\Twt-CBOW\\Twt-CBOW')
        else :
            self.embeddings = KeyedVectors.load_word2vec_format(path+'\\GoogleNews-vectors-negative300.bin', binary=True,limit=100000000) 
        
    def fit_transform(self, data, y=None):
        #if bArabic:
        #    embeddings = gensim.models.Word2Vec.load(path+'\\Twt-CBOW\\Twt-CBOW')
        #else :
        #    embeddings = KeyedVectors.load_word2vec_format(path+'\\GoogleNews-vectors-negative300.bin', binary=True,limit=100000000) 
        docs_vectors = pd.DataFrame() # creating empty final dataframe
        for doc in data:
            temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
            for word in doc.split(' '): # looping through each word of a single document and spliting through space
                try:
                    word_vec = self.embeddings.wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    #word_vec.fillna(0)
                    temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
                except:
                    pass    
            doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
            #print(embeddings)
            docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
        
        return docs_vectors
    def transform(self, data):
        return self.fit_transform(data) 

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

def TrainAll(IsApriori = 'F',IsArabic=False):
    #if bArabic:
    #    embeddings = gensim.models.Word2Vec.load(path+'\\Twt-CBOW\\Twt-CBOW')
    #else :
    #    embeddings = KeyedVectors.load_word2vec_format(path+'\\GoogleNews-vectors-negative300.bin', binary=True,limit=100) 
    
# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
#model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
    #w2v = {w: vec for w, vec in zip(embeddings.wv.index2word, embeddings.wv.syn0)}
    #w2v = dict(zip(embeddings.wv.index2word, embeddings.wv.syn0))
    #print(w2v)
    vectorizerList = [[CountVectorizer(),'CountVectorizer'],[ TfidfVectorizer(),'TfidfVectorizer'],[EmbeddingVectorizer(IsArabic),'EmbeddingVectorizer']]
    clfrList = [[naive_bayes.MultinomialNB(),'MultinomialNB'],[LogisticRegression(),'LogisticRegression'],
                [KNeighborsClassifier(),'KNeighborsClassifier'],[RandomForestClassifier(),'RandomForestClassifier'],
                [SVC(),'SVC'],[LinearSVC(),'LinearSVC'],[DecisionTreeClassifier(),'DecisionTreeClassifier'],
                [MLPClassifier(),'MLPClassifier']]
    

    for vectorizer,vectorizerName in vectorizerList:
        for clfr,clfrName in clfrList : 
            if not ((clfrName =='MultinomialNB' or not IsArabic) and vectorizerName=='EmbeddingVectorizer'):
                Train(data=preProcessed , vectorizer=vectorizer ,clfr=clfr ,n_splits=10, fileName=clfrName+' ==> '+vectorizerName, IsApriori = 'F',IsArabic= IsArabic)


In [77]:
#Train Data
bArabic = True
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=False, IsArabic=bArabic)
TrainAll('F',bArabic)


------------MultinomialNB ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.87      0.97      0.92       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and Executive Programs (ED)       1.00      0.75      0.86         4
          Forums and Conferences (FC)       1.00      0.67      0.80         6
             Functional Programs (FN)       0.85      0.57      0.69        40

                          avg / total       0.87      0.87      0.86       190

------------LogisticRegression ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.90      0.96      0.93       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and E

------------DecisionTreeClassifier ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.93      0.83      0.88       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and Executive Programs (ED)       1.00      0.75      0.86         4
          Forums and Conferences (FC)       0.80      0.67      0.73         6
             Functional Programs (FN)       0.56      0.80      0.66        40

                          avg / total       0.84      0.82      0.82       190

------------MLPClassifier ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.89      0.96      0.92       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education a

In [78]:
bArabic = False
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=False, Root=True, IsArabic=bArabic)
TrainAll('F',bArabic)

------------MultinomialNB ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.85      0.79      0.82       107
                 Computer Skills (CS)       0.84      0.88      0.86        43
Core and Managerial Competencies (CM)       0.89      0.88      0.88        88
Education and Executive Programs (ED)       0.81      0.88      0.84       100
          Forums and Conferences (FC)       0.00      0.00      0.00         6
             Functional Programs (FN)       0.89      0.89      0.89       484
                        Language (LN)       1.00      1.00      1.00        20

                          avg / total       0.87      0.87      0.87       848

------------LogisticRegression ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

               

------------RandomForestClassifier ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.83      0.79      0.81       107
                 Computer Skills (CS)       0.93      0.91      0.92        43
Core and Managerial Competencies (CM)       0.88      0.91      0.89        88
Education and Executive Programs (ED)       0.93      0.85      0.89       100
          Forums and Conferences (FC)       0.00      0.00      0.00         6
             Functional Programs (FN)       0.89      0.93      0.91       484
                        Language (LN)       1.00      1.00      1.00        20

                          avg / total       0.89      0.89      0.89       848

------------SVC ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                     

In [79]:
#Train Data using Apriori
bArabic = True
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=False, IsArabic=bArabic)

TrainAll(IsApriori = 'T',IsArabic=bArabic)

------------MultinomialNB ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.87      0.98      0.92       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and Executive Programs (ED)       1.00      0.75      0.86         4
          Forums and Conferences (FC)       1.00      0.50      0.67         6
             Functional Programs (FN)       0.85      0.57      0.69        40

                          avg / total       0.87      0.87      0.86       190

------------LogisticRegression ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.88      0.96      0.92       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and E

------------LinearSVC ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.85      0.97      0.91       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and Executive Programs (ED)       1.00      0.75      0.86         4
          Forums and Conferences (FC)       1.00      0.67      0.80         6
             Functional Programs (FN)       0.84      0.53      0.65        40

                          avg / total       0.85      0.86      0.84       190

------------DecisionTreeClassifier ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.94      0.87      0.90       139
Core and Managerial Competencies (CM)       0.00      0.00      0.00         1
Education and E

In [80]:
bArabic = False
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=False, Root=True, IsArabic=bArabic)

TrainAll(IsApriori = 'T',IsArabic=bArabic)

------------MultinomialNB ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.84      0.79      0.82       107
                 Computer Skills (CS)       0.84      0.88      0.86        43
Core and Managerial Competencies (CM)       0.89      0.88      0.88        88
Education and Executive Programs (ED)       0.82      0.88      0.85       100
          Forums and Conferences (FC)       0.00      0.00      0.00         6
             Functional Programs (FN)       0.89      0.90      0.89       484
                        Language (LN)       1.00      1.00      1.00        20

                          avg / total       0.87      0.88      0.87       848

------------LogisticRegression ==> CountVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

               

------------RandomForestClassifier ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                       Awareness (AW)       0.86      0.82      0.84       107
                 Computer Skills (CS)       0.89      0.93      0.91        43
Core and Managerial Competencies (CM)       0.85      0.85      0.85        88
Education and Executive Programs (ED)       0.96      0.85      0.90       100
          Forums and Conferences (FC)       0.50      0.17      0.25         6
             Functional Programs (FN)       0.90      0.93      0.91       484
                        Language (LN)       1.00      1.00      1.00        20

                          avg / total       0.89      0.90      0.89       848

------------SVC ==> TfidfVectorizer---------Average 10 Folds----------------------
                                       precision    recall  f1-score   support

                     

In [26]:

#preProcessed_ItemSet = (preProcessed["CourseName"]+ " "+preProcessed["Category"].strip()).apply(lambda x : x.split(' '))

27     [توعيه, عامل, مخاطر, كهربيه, CAT:Awareness (AW)]
31    [اجراء, سلامه, صحه, مهنيه, لعامل, مرافق, اثناء...
33    [توعيه, عامل, باعداد, درفله, تعليم, سلامه, صحه...
34    [توعيه, عامل, بخط, انتاج, اطوال, تعليم, سلامه,...
35    [توعيه, عامل, تسليم, اتباع, تعليم, سلامه, صحه,...
dtype: object

time_elapsed: 0.0007234533627827962 minutes


In [38]:
#generate Decision tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz

bArabic = False
preProcessed = PreProcessing(ds1_train,removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=False, IsArabic=bArabic)

train_data = preProcessed["CourseName"]
train_label = preProcessed["Category"]
#    vectorizerList = [[CountVectorizer(),'CountVectorizer'],[ TfidfVectorizer(),'TfidfVectorizer'],[EmbeddingVectorizer(IsArabic),'EmbeddingVectorizer']]
#    clfrList = [[naive_bayes.MultinomialNB(),'MultinomialNB'],[LogisticRegression(),'LogisticRegression'],
#                [KNeighborsClassifier(),'KNeighborsClassifier'],[RandomForestClassifier(),'RandomForestClassifier'],
#               [SVC(),'SVC'],[LinearSVC(),'LinearSVC'],[DecisionTreeClassifier(),'DecisionTreeClassifier'],
#                [MLPClassifier(),'MLPClassifier']]
vectorizer =CountVectorizer()
clfr =  DecisionTreeClassifier()
xtrain = vectorizer.fit_transform(train_data)
#print(vectorizer.get_feature_names())
        #xtrain.fillna(0)
        #xval.fillna(0)
clfr.fit(xtrain,train_label)
print(clfr.classes_)
print(vectorizer.get_feature_names())
#predicted = clfr.predict(xval)


['Awareness (AW)' 'Computer Skills (CS)'
 'Core and Managerial Competencies (CM)'
 'Education and Executive Programs (ED)' 'Forums and Conferences (FC)'
 'Functional Programs (FN)' 'Language (LN)']
['10g', '11g', '11i', '4g', '5s', 'ac', 'acceler', 'access', 'accord', 'account', 'accredit', 'activ', 'actuat', 'adjust', 'administ', 'administr', 'advanc', 'aid', 'air', 'align', 'alloy', 'altern', 'amend', 'analysi', 'analyt', 'analyz', 'appli', 'applic', 'apprais', 'arc', 'asnt', 'assess', 'assessor', 'asset', 'associ', 'assur', 'audit', 'auditor', 'autocad', 'autom', 'automat', 'awar', 'bachelor', 'balanc', 'base', 'basic', 'bear', 'beginn', 'behavior', 'belaru', 'belt', 'benefit', 'boiler', 'box', 'bsci', 'build', 'busi', 'cabl', 'calcul', 'calibr', 'carbon', 'cardiopulmonari', 'case', 'cast', 'castolin', 'casttolin', 'categori', 'cathod', 'caus', 'ccm', 'cdc', 'celemi', 'center', 'central', 'centrifug', 'certif', 'certifi', 'cfr', 'chain', 'chemic', 'cia', 'circl', 'circuit', 'cisa', 

In [36]:
        
dot_data = StringIO()
export_graphviz(clfr, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = vectorizer.get_feature_names(),class_names=['Awareness (AW)', 'Core and Managerial Competencies (CM)',
 'Education and Executive Programs (ED)', 'Forums and Conferences (FC)',
 'Functional Programs (FN)'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(path+"\DecissionTree.png")
Image(graph.create_png())

AttributeError: module 'pydotplus' has no attribute 'Node'

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,IsCategory
9800841,(تعليم),(CAT:Awareness (AW)),0.447368,0.731579,0.447368,1.000000,1.366906,0.120083,inf,True
9800852,(سلامه),(CAT:Awareness (AW)),0.457895,0.731579,0.442105,0.965517,1.319772,0.107119,7.784211,True
9800840,(صحه),(CAT:Awareness (AW)),0.436842,0.731579,0.426316,0.975904,1.333969,0.106731,11.139474,True
9800802,(مهنيه),(CAT:Awareness (AW)),0.431579,0.731579,0.426316,0.987805,1.350237,0.110582,22.010526,True
9800797,"(صحه, مهنيه)",(CAT:Awareness (AW)),0.431579,0.731579,0.426316,0.987805,1.350237,0.110582,22.010526,True
9800835,"(صحه, سلامه)",(CAT:Awareness (AW)),0.431579,0.731579,0.421053,0.975610,1.333567,0.105319,11.005263,True
9800791,"(سلامه, مهنيه)",(CAT:Awareness (AW)),0.426316,0.731579,0.421053,0.987654,1.350031,0.109169,21.742105,True
9800777,"(سلامه, صحه, مهنيه)",(CAT:Awareness (AW)),0.426316,0.731579,0.421053,0.987654,1.350031,0.109169,21.742105,True
9800844,"(تعليم, سلامه)",(CAT:Awareness (AW)),0.378947,0.731579,0.378947,1.000000,1.366906,0.101717,inf,True
9800741,"(تعليم, سلامه, مهنيه)",(CAT:Awareness (AW)),0.368421,0.731579,0.368421,1.000000,1.366906,0.098892,inf,True


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,IsCategory


In [185]:
#pip install hunspell
#pip install aspell-python-py3

#pip install aspell-ar
#pip install aspell-ar-large

In [184]:
#spell Checker
from spellchecker import SpellChecker

spell = SpellChecker(language='ar')  # Arabic dictionary
print(spell['السلام'])

ValueError: The provided dictionary language (ar) does not exist!

In [ ]:
preProcessed.shape

In [ ]:
docs_vectors.head()